# Project Premature Stop Codons in *Arabidopsis thaliana*



To manage live in an ecosystem organisms need to adapt to the specific abiotic and biotic factors arount them. Especially for plants adaptation is important because they can not change their habitat during their lifespan. Adaptation can happen through multiple changes on cell, tissue and organism level, but the fundamental process that drive adaptation on the smallest scale are mutations. There are three categories of mutations, which have different effects on the process of transkription of DNA to RNA and the translation from the RNA to the protein. The first category of mutations are synonymous mutations. **TODO ongoing introduction**

## Ara11-Annotation

### Analysing positions of gene locations 

For determining length or position of genes or RNAs we need information about the start of the 5'UTR, the start of the coding sequence, the stop of the coding sequence, the stop of the 3' UTR and the orientation of the gene on the DNA strand for each single gene in *A. thaliana*. For getting a detailed look at the different genes we actually look at the start  and stop of the protein as well as the UTR regions. This leads to multiple entries for different splice variants for one gene. Different splicing variants result in different protein start and stop and can also result in different 5'UTR and 3'UTR regions depending on the annotation in the Araport 11 file. We store all genes, which don't have a protein as a result for example miRNAs in a different table. For them only the information of where the gene starts and ends is available. 

In [ ]:
import pandas as pd

In [ ]:
%run Ara11_Annotation.py

## Premature-Stop Attributes

## Control with Synonymous and Non-Synonymous Mutations